# Waste-to-energy problem (julia)



$$ 
\left \{
		\begin{array}{lll}
			\textcolor{white}{\dot{x}(t)} = \omega - \big(\beta + \textcolor{white}{K(t)}\textcolor{red}{q(t)} \big) \textcolor{white}{x(t)} \\[2mm]
			\textcolor{white}{\dot{K}(t)} = \textcolor{red}{I(t)} - \gamma \textcolor{white}{K(t)}\\[2mm]
			\textcolor{white}{\dot{E}(t)} = \mu \textcolor{white}{K(t)x(t)}\textcolor{red}{q(t)} -   (\alpha + n \textcolor{white}{K(t)}) \textcolor{white}{E(t)}
		\end{array}
\right.	
$$

$\textbf{[OCP]}$ Find admissible controls $\textcolor{red}{(I,q)}$ maximizing,
$$
		J : =\int_{0}^{\textcolor{magenta}{T}}  e^{-\delta t} \textcolor{violet}{\mathcal{J}(t)} dt
$$
$$
	    \textcolor{violet}{\mathcal{J}(t) = p \textcolor{white}{E(t)} - c \textcolor{white}{K(t)x(t)}\textcolor{red}{q(t)} - \textcolor{red}{I(t)}(c_1 + c_2 \textcolor{red}{I(t)})}
$$

over a finite fixed-time horizon $[0,T]$,  with $ \delta,p,c,c_1,c_2 \geq 0$.

## Direct Solve

In [1]:
using JuMP, Ipopt, Plots, MINPACK
#JuMP model, Ipopt solver
sys = Model(optimizer_with_attributes(Ipopt.Optimizer,"print_level"=> 5))
set_optimizer_attribute(sys,"tol",1e-10)
set_optimizer_attribute(sys,"constr_viol_tol",1e-10)
set_optimizer_attribute(sys,"max_iter",5000)

# Model Parameters
w       = 50.0 
b       = 0.25
a       = 0.1
g       = 0.2
d       = 0.1
mu      = 0.8
n       = 0.015  # n can be zero
p       = 1.0
c       = 2.0  
c1      = 2.0
c2      = 3.0
N       = 1000
t0      = 0.0
tf      = 10.0 
x0      = 1.0
k0      = 1.0
E0      = 0.0
cr0     = 0.0
Imax    = 5.0  # Upper bound on control 1
qmax    = 1.0  # Upper bound on control 2

xi0 = [ x0, k0, E0, cr0]                #initial conditions IC
dt=tf/N;                                #defining time-step


In [2]:
# Variables (some state constraints have been added to ease convergence)
@variables(sys, begin
    x[1:N+1] >= 0.0                     #x
    k[1:N+1] >= 0.0                     #k
    E[1:N+1] >= 0.0                     #E
    cr[1:N+1]                           #Extra-state 
    0.0<= q[1:N+1]<= qmax               #q
    0.0<= I[1:N+1]<= Imax               #I
end)

# Objective
@objective(sys, Max, cr[N+1])           #maximize the criteria over a finite time-horizon 
#Remark: maximize the final-state of the extra-state Cr(Tf)=Cr[N+1]


# Boundary constraints 
@constraints(sys, begin
    con_x0, x[1]    == x0
    con_k0, k[1]    == k0
    con_E0, E[1]    == E0
    con_cr0, cr[1]  == cr0 
end)

# Dynamics
@NLexpressions(sys, begin
    dx[i= 1:N+1], w- (b+q[i]*k[i])*x[i]
    dk[i= 1:N+1], I[i]-g*k[i]
    dE[i= 1:N+1], mu*q[i]*x[i]*k[i] - (a+n*k[i])*E[i]
    dcr[i= 1:N+1], exp(-d*dt*(i-1))*(p*E[i]-c*q[i]*x[i]*k[i]- I[i]*(c1+c2*I[i]))      
end)

# Crank-Nicolson scheme
@NLconstraints(sys, begin
    con_dx[i =1:N], x[i+1] == x[i] + dt * (dx[i] + dx[i+1])/2.0 
    con_dk[i =1:N], k[i+1] == k[i] + dt * (dk[i] + dk[i+1])/2.0 
    con_dE[i =1:N], E[i+1] == E[i] + dt * (dE[i] + dE[i+1])/2.0 
    con_dcr[i =1:N], cr[i+1] == cr[i] + dt * (dcr[i] + dcr[i+1])/2.0 
end);

In [3]:
# Solver for the control and state for the direct method [OUTPUT PRINT]
println("Solving...")
status = optimize!(sys)

if termination_status(sys) == MOI.OPTIMAL
    println("Solution is optimal")
elseif termination_status(sys) == MOI.LOCALLY_SOLVED
    println("Local solution found")
elseif termination_status(sys) == MOI.TIME_LIMIT && has_values(sys)
    println("Solution is suboptimal due to a time limit, but a primal solution is available")
else
    error("The model was not solved correctly.")
end
println("Objective value = ", objective_value(sys), "\n")

In [ ]:
#Optimal state trajectories, optimal co-states and optimal controls I*,q*
t   = (1:N+1)*dt; t = (t[1:end-1] + t[2:end])/2.0
xx  = value.(x); xx = (xx[1:end-1] + xx[2:end])/2.0
kk  = value.(k); kk = (kk[1:end-1] + kk[2:end])/2.0
EE  = value.(E); EE = (EE[1:end-1] + EE[2:end])/2.0
crr = value.(cr); crr = (crr[1:end-1] + crr[2:end])/2.0
qq  = value.(q); qq = (qq[1:end-1] + qq[2:end])/2.0
II  = value.(I); II= (II[1:end-1] + II[2:end])/2.0
xi  =  [ [ xx[i], kk[i], EE[i], crr[i] ] for i in 1:N ]
pp  = -[ [dual(con_dx[i]),dual(con_dk[i]), dual(con_dE[i]), dual(con_dcr[i]) ] for i in 1:N ];

In [ ]:
#plot the optimal states and costates

x_plot = plot(t, [ xi[i][1] for i in 1:N ], xlabel = "t", ylabel = "x", legend = false)
k_plot = plot(t, [ xi[i][2] for i in 1:N ], xlabel = "t", ylabel = "k", legend = false)
e_plot = plot(t, [ xi[i][3] for i in 1:N ], xlabel = "t", ylabel = "E", legend = false)
cr_plot = plot(t, [ xi[i][4] for i in 1:N ], xlabel = "t", ylabel = "cr", legend = false)
px_plot = plot(t, [ pp[i][1] for i in 1:N ], xlabel = "t", ylabel = "px", legend = false)
pk_plot = plot(t, [ pp[i][2] for i in 1:N ], xlabel = "t", ylabel = "pk", legend = false)
pe_plot = plot(t, [ pp[i][3] for i in 1:N ], xlabel = "t", ylabel = "pE", legend = false)
pcr_plot = plot(t, [ pp[i][4] for i in 1:N ], xlabel = "t", ylabel = "pcr", legend = false)
xi_plot = plot(x_plot, px_plot, k_plot, pk_plot, e_plot, pe_plot,cr_plot,pcr_plot, layout = (4,2),size=(800, 1200))
display(xi_plot)

In [ ]:
############ Some extra-plots of key functions : Optimal control analysis
function hdag(t,lmd)
    Is= (exp(d*t)*lmd[2]-c1)/ (2*c2)
    return Is
end
function swq(t,lmd)
    sw= - c - exp(d*t)*lmd[1] + mu* exp(d*t)*lmd[3]        #Pseudoswitch_function
    return sw
end
q_plot = plot(t, qq, xlabel = "t", ylabel = "q", legend = false)
I_plot = plot(t, II, xlabel = "t", ylabel = "I", legend = false)
sw_plot= plot(t, swq.(t,pp), xlabel = "t", ylabel = "switch function", legend = false)
hdag_plot= plot(t, hdag.(t,pp), xlabel = "t", ylabel = "hdag", legend = false)

plot(sw_plot,hdag_plot,q_plot,I_plot,layout = (2,2),size=(800, 400))

The current value Hamiltonian:

$$
\begin{array}{lll}
    \textcolor{nred}{H =} & \textcolor{nred}{h(X,\Lambda) + \tilde{h} Kx \textcolor{cyan}{q} + h^\dag(\textcolor{cyan}{I})}, \\
\end{array}
$$
with,
$$h(X,\Lambda)= pE + \lambda_1 (w-\beta x) -\lambda_2\gamma K - \lambda_3\alpha E$$


$$\tilde{h} = - c - \lambda_1 + \mu\lambda_3$$


$$h^\dag(I) = -I(c_1+c_2I) + \lambda_2I$$

The pseudo-costates are given by,


$$\left\{
\begin{array}{lll}
    & \textcolor{nred}{\dot{\lambda}_1 = (\delta+\beta)\lambda_1 -\tilde{h}Kq} \\[2mm]
    &  \dot{\lambda}_2 = (\delta+\gamma)\lambda_2 -\tilde{h}xq   \\[2mm]
    &  \textcolor{nred}{\dot{\lambda}_3 = (\delta+\alpha)\lambda_3-p  } 
\end{array}
\right.
$$

The transversality conditions are given by,

$$
    \textcolor{white}{\lambda_i(T)=0, \:\:\: \forall i=1,2,3}
$$

The optimal control $\textcolor{nred}{I^*(t)}$ satisfies,
$$
    \textcolor{nred}{I^*(t) =
        \left\{
        \begin{array}{lll}
             & 0 \:\:
             \text{if} \: \lambda_2(t) \leq c_1\\[3mm]
             & \min \left\{ \frac{\lambda_2(t)-c_1}{2c_2}, I_{\max} \right\} \:\: \text{if} \: \lambda_2(t) > c_1
        \end{array}
        \right.}
$$

The optimal control \textcolor{nred}{$q^*(t)$} satisfies,
$$
    \textcolor{nred}{q^*(t) =
        \left\{
        \begin{array}{ll}
             & 0, \:\:
             \text{if} \: \tilde{h} = - c - \lambda_1 + \mu\lambda_3 <0\\[1.5mm]
             & q_{max} \:\:
             \text{if} \: \tilde{h} >0,\\[1.5mm]
        \end{array}
        \right.}
$$

In [ ]:
include("flow.jl");

In [ ]:
#Hamiltonian def

function h0(t,xi,lmd)                   #h0 is when both control are zero (i.e q=0 and I=0)
    x, k, E, cr= xi
    px, pk, pE, pcr= lmd
    q   = 0.0
    I   = 0.0
    r   = pcr*exp(-d*t)*(p*E-c*q*x*k- I*(c1+c2*I))+px*(w-(b+q*k)*x)+pk*(I-g*k)+pE*(mu*q*x*k-(a+n*k)*E)
    return r
end

function h10(t,xi,lmd)                   #h10 is when q=1 and I=0
    x, k, E, cr= xi
    px, pk, pE, pcr= lmd
    q   = 1.0
    I   = 0.0
    r   = pcr*exp(-d*t)*(p*E-c*q*x*k- I*(c1+c2*I))+px*(w-(b+q*k)*x)+pk*(I-g*k)+pE*(mu*q*x*k-(a+n*k)*E)
    return r
end

function h1s(t,xi,lmd)                    #h1s is when q=1 and I=Is           
    x, k, E, cr= xi
    px, pk, pE, pcr= lmd
    q   = 1.0
    I   = (exp(d*t)*pk-c1)/(2*c2)
    r   = pcr*exp(-d*t)*(p*E-c*q*x*k- I*(c1+c2*I))+px*(w-(b+q*k)*x)+pk*(I-g*k)+pE*(mu*q*x*k-(a+n*k)*E)
    return r
end

######################################################################################################
f0      = Flow(h0);
f10     = Flow(h10);
f1s     = Flow(h1s);
######################################################################################################

#Shooting function

function shoot(p0, ti, tq)  # p0 initial costate, ti: switching instant control-I, tq: switching instant control-q
    xs,ps  = f1s(t0, xi0, p0, ti)
    xq,ppq = f10(ti, xs, ps ,tq)
    xf,pf  = f0(tq, xq, ppq, tf)
    s      = zeros(eltype(p0),6)
    s[1:4] = pf[1:4] - [0.0, 0.0, 0.0, 1.0]
    s[5]   = hdag(ti,ps)  # hdag: (like)switching function in I
    s[6]   = swq(tq,ppq) # swq : switching function in q
    return s;
end

In [ ]:
display(plot(q_plot, sw_plot, I_plot, hdag_plot, layout = (2,2)))
η    = 1e-3
p0   = pp[1]
ti1  = t[hdag.(t, pp) .≤ η ]
tq1  = t[swq.(t ,pp) .≤ η ]
ti   = min(ti1...)
tq   = min(tq1...)
ξ    = [p0 ; ti ; tq]
println("Initial guess:\n", ξ)

In [ ]:
# Solve
foo(ξ)          = shoot(ξ[1:4], ξ[5], ξ[6])
jfoo(ξ)         = ForwardDiff.jacobian(foo, ξ)
foo!(s, ξ)      = ( s[:] = foo(ξ); nothing )
jfoo!(js, ξ)    = ( js[:] = jfoo(ξ); nothing )

println("Initial value of shooting:\n", foo(ξ), "\n\n")

#nl_sol = fsolve(foo!, jfoo!, ξ, show_trace=true); println(nl_sol)
nl_sol = fsolve(foo!, ξ, show_trace=true); println(nl_sol)

# Retrieves solution
if nl_sol.converged
    p0 = nl_sol.x[1:4]
    ti = nl_sol.x[5]
    tq = nl_sol.x[6];
else
    error("Not converged")
end

In [ ]:
# Plots

ode_sol = f1s((t0, ti), xi0, p0)
tt0 = ode_sol.t
xx0 = [ ode_sol[1:4, j] for j in 1:size(tt0, 1) ]
pp0 = [ ode_sol[5:8, j] for j in 1:size(tt0, 1) ]
qq0 = ones(size(tt0, 1))
ii0= hdag.(tt0, pp0)
###################################
ode_sol = f10((ti, tq), xx0[end], pp0[end])
tt1 = ode_sol.t
xx1 = [ ode_sol[1:4, j] for j in 1:size(tt1, 1) ]
pp1 = [ ode_sol[5:8, j] for j in 1:size(tt1, 1) ]
qq1 = ones(size(tt1, 1))
ii1= zeros(size(tt1, 1))

########################################
ode_sol = f0((tq, tf), xx1[end], pp1[end])
tt2 = ode_sol.t
xx2 = [ ode_sol[1:4, j] for j in 1:size(tt2, 1) ]
pp2 = [ ode_sol[5:8, j] for j in 1:size(tt2, 1) ]
qq2 = zeros(size(tt2,1))
ii2= zeros(size(tt2,1));

In [ ]:

tt=[tt0; tt1; tt2]
xxi=[xx0; xx1; xx2]
ppp=[pp0; pp1; pp2]
qqq=[qq0;qq1; qq2]
iii=[ii0; ii1; ii2]
N=length(tt)

xx_plot = plot(tt, [ xxi[i][1] for i in 1:N ], xlabel = "t", ylabel = "x", legend = false)
kk_plot = plot(tt, [ xxi[i][2] for i in 1:N ], xlabel = "t", ylabel = "k", legend = false)
ee_plot = plot(tt, [ xxi[i][3] for i in 1:N ], xlabel = "t", ylabel = "E", legend = false)
crcr_plot = plot(tt, [ xxi[i][4] for i in 1:N ], xlabel = "t", ylabel = "cr", legend = false)
qqq_plot = plot(tt, qqq, xlabel = "t", ylabel = "qindirect", legend = false)
iii_plot = plot(tt, iii, xlabel = "t", ylabel = "qindirect", legend = false)
ppx_plot = plot(tt, [ ppp[i][1] for i in 1:N ], xlabel = "t", ylabel = "px", legend = false)
ppk_plot = plot(tt, [ ppp[i][2] for i in 1:N ], xlabel = "t", ylabel = "pk", legend = false)
ppe_plot = plot(tt, [ ppp[i][3] for i in 1:N ], xlabel = "t", ylabel = "pE", legend = false)
ppcr_plot = plot(tt, [ ppp[i][4] for i in 1:N ], xlabel = "t", ylabel = "pcr", legend = false)
xxi_plot = plot(xx_plot, ppx_plot, kk_plot, ppk_plot, ee_plot, ppe_plot,crcr_plot, ppcr_plot, layout = (4,2),size=(800, 1200))
display(xxi_plot)


In [ ]:
sw2_plot= plot(tt, swq.(tt, ppp), xlabel = "t", ylabel = "switch function", legend = false)
display(plot(sw2_plot,qqq_plot,layout=(2,1)))

uI_plot= plot(tt, hdag.(tt, ppp), xlabel = "t", ylabel = "hdag", legend = false)
display(plot(iii_plot,uI_plot,layout=(2,1)))